<a href="https://colab.research.google.com/github/LennyTheIntern/valorant_tflite_model/blob/master/quant_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
saved_model_dir  ='Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport/saved_model'
tflite_model_quant_file = 'tpu_detect.tflite'
img_dir = 'Tensorflow/workspace/images/train/'
trainrec = 'Tensorflow/workspace/annotations/train.record'
testrec = 'Tensorflow/workspace/annotations/test.record'

def data_gen():
    image_arr = []
    for i in range(160):
        inst = anns[i]
        file = inst['filename']
        img = cv2.imread(img_dir + file)
        img = cv2.resize(img, (NORM_H, NORM_W)) # height and width of the image
        img = img / 255.0
        img = img.astype(np.float32)  
        a.append(img)
    image_arr = np.array(a)
    print(a.shape) # a is np array of 160 3D images
    img = tf.data.Dataset.from_tensor_slices(image_arr).batch(1) # the more files you have the larger the batch size should be
    for i in img.take(BATCH_SIZE):
        print(i)
        yield [i]

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

In [ ]:
!edgetpu_compiler tpu_detect.tflite